# Joins

- Explore new schemas
- Cross Joins
- Natural Joins and "Semantic joins"
- Equijoins
- `SQL: NATURAL JOIN, USING, and ON`

In [ ]:
# setup SQL magic 

import json
with open('cred.json') as f:
    creds = json.load(f)

connection_string = "mysql://{user}:{password}@{host}".format(**creds)

import pymysql 
pymysql.install_as_MySQLdb()

%load_ext sql
%config SqlMagic.autocommit=True
%sql $connection_string

In [ ]:
import datajoint as dj

In [ ]:
dj.list_schemas()

In [ ]:
schema = dj.Schema('shared_airport')
schema = dj.Schema('shared_sakila')
schema = dj.Schema('shared_world')
schema = dj.Schema('shared_airport')


In [ ]:
# create classes for working with tables
schema.spawn_missing_classes()

In [ ]:
dj.Diagram(schema)

In [ ]:
Passenger()

In [ ]:
%%sql

SHOW SCHEMAS

In [ ]:
%%sql

USE shared_sales

In [ ]:
%%sql

SHOW TABLES

In [ ]:
%%sql

SHOW CREATE TABLE category;

In [ ]:
%%sql

-- show all customers from Texas 

SELECT *
FROM customer
WHERE customer_state = "TX"


In [ ]:
%%sql

-- show the names and addresses of all Texas customers sorted by last name 

SELECT CONCAT(customer_first_name, ' ', customer_last_name) AS full_name, 
    customer_street_address AS address
  FROM customer
  WHERE customer_state = "TX"
  ORDER BY customer_last_name;

```sql
-- show the names and addresses of all Texas customers sorted by last name 

SELECT CONCAT(customer_first_name, ' ', customer_last_name) AS full_name, 
    customer_street_address AS address
  FROM customer
  WHERE customer_state = "TX"
  ORDER BY customer_last_name;
```

In [ ]:
%%sql

-- show the name of the most expensive product 
-- A bad query.

SELECT product_name, max(retail_price) as max_price
FROM product

In [ ]:
%%sql

-- show the name of the most expensive product 

SELECT product_name, retail_price
FROM product
ORDER BY retail_price DESC
LIMIT 1

In [ ]:
%%sql

-- show all products above 10 in quantity and discount them by 10%

SELECT product_number, product_name, retail_price * 0.9 as retail_price
FROM product
WHERE quantity_on_hand > 10


In [ ]:
%%sql

-- show all orders

SELECT * FROM `order`;

In [ ]:
%%sql
SHOW CREATE TABLE `order`

## Same queries in DataJoint

In [ ]:
# show all customers from Texas 
Customer() & 'customer_state = "TX"'

In [ ]:
Customer() & {'customer_state': "TX"}

In [ ]:
# show the names and addresses of all Texas customers sorted by last name 

texas_customers = Customer() &  {'customer_state': "TX"}

q = texas_customers.proj('customer_first_name', 'customer_last_name')

q.fetch(order_by='customer_last_name', as_dict=True)

In [ ]:
# show the name of the most expensive product 
Product().proj('product_name', 'retail_price').fetch(
    'product_name', limit=1, order_by='retail_price DESC')[0]

In [ ]:
# show all products above 10 in quantity and discount them by 10%
(Product() & 'quantity_on_hand > 10').proj(
    'product_name', retail_price='retail_price * 0.9')

## Queries with subqueries in restriction

In [ ]:
dj.Diagram(schema)

In [ ]:
# Give me all orders for Texas customers

texas_customers = Customer() & {'customer_state': 'TX'}
Order & texas_customers

In [ ]:
%%sql

SELECT customer_id 
FROM customer
WHERE customer_state = "TX"

In [ ]:
%%sql

-- orders placed by Texas customers

SELECT * FROM `order`
WHERE customer_id IN (
    SELECT customer_id 
    FROM customer
    WHERE customer_state = "TX"
)

In [ ]:
Product()

In [ ]:
%%sql 
-- select all orders containing product_number 1

SELECT * FROM `order` WHERE order_number IN (
SELECT order_number FROM `order__item` WHERE product_number = 1)


In [ ]:
Order & (Order.Item & 'product_number=1')

In [ ]:
# Show all customers who have never placed an order

In [ ]:
%%sql
SELECT * FROM customer WHERE customer_id NOT IN 
(SELECT customer_id FROM `order`)

In [ ]:
Customer - Order

In [ ]:
%%sql 
-- products with the word "Bike" in their names
SELECT * FROM product WHERE product_name LIKE "%Bike%"